In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import re

In [4]:
# 学習用データの読み込み
df = pd.read_csv('drive/My Drive/signate/student_cup/dataset/train.csv')
dftest = pd.read_csv('drive/My Drive/signate/student_cup/dataset/test.csv')

print(df.shape)
print(dftest.shape)

(31470, 17)
(31262, 16)


In [0]:
df_x = pd.DataFrame()
dftest_x = pd.DataFrame()

# 賃料（目的変数）

In [5]:
df_t = pd.DataFrame(df['賃料'])

df_t.head()

,賃料
0,75000
1,76000
2,110000
3,150000
4,74000


# 所在地

In [6]:
df_temp = df['所在地']

df_temp = df_temp.str.extract('(.*都)(.*区)(.*)(.+)丁目(.*)', expand=True)
df_temp.columns = ['0', '1', '2', '3', '4']
df_temp['4'] = df_temp['4'].replace('', '-')
df_temp['4'] = df_temp['4'].fillna('-')
df_location = pd.concat(objs=[df_temp, pd.DataFrame(df_temp['4'].str.split('-').to_list())], axis=1)
df_location = df_location.drop('4', axis=1)
df_location.columns = ['prefecture', 'city', 'town', 'chome', 'street', 'house', 'mistake']

df_location.loc[7088, 'street'] = 6
df_location.loc[7088, 'house'] = 2
df_location.loc[19365, 'street'] = 6 
df_location.loc[19365, 'house'] = 2
df_location.loc[22883, 'street'] = 1 
df_location.loc[22883, 'house'] = 17

df_location = df_location.drop('mistake', axis=1)

df_location.head()

,prefecture,city,town,chome,street,house
0,東京都,北区,滝野川,３,,
1,東京都,中央区,月島,３,,
2,東京都,渋谷区,笹塚,２,,
3,東京都,杉並区,高円寺南,２,23,2
4,東京都,葛飾区,金町,３,7,2


In [7]:
dftest_temp = dftest['所在地']

dftest_temp = dftest_temp.str.extract('(.*都)(.*区)(.*)(.+)丁目(.*)', expand=True)
dftest_temp.columns = ['0', '1', '2', '3', '4']
dftest_temp['4'] = dftest_temp['4'].replace('', '-')
dftest_temp['4'] = dftest_temp['4'].fillna('-')
dftest_location = pd.concat(objs=[dftest_temp, pd.DataFrame(dftest_temp['4'].str.split('-').to_list())], axis=1)
dftest_location = dftest_location.drop('4', axis=1)
dftest_location.columns = ['prefecture', 'city', 'town', 'chome', 'street', 'house', 'mistake']

dftest_location.loc[3048, 'street'] = 8
dftest_location.loc[3048, 'house'] = 16
dftest_location.loc[3154, 'street'] = 41 
dftest_location.loc[3154, 'house'] = 12
dftest_location.loc[14392, 'street'] = 2 
dftest_location.loc[14392, 'house'] = 2
dftest_location.loc[24731, 'street'] = 9 
dftest_location.loc[24731, 'house'] = 20

dftest_location = dftest_location.drop('mistake', axis=1)

dftest_location.head()

,prefecture,city,town,chome,street,house
0,東京都,世田谷区,深沢,５,1,27
1,東京都,目黒区,八雲,１,11,8
2,東京都,豊島区,池袋本町,２,22,2
3,東京都,杉並区,和泉,１,,
4,東京都,杉並区,堀ノ内,２,,


In [0]:
#1

# アクセス

In [9]:
df_access = df['アクセス']

df_access = pd.DataFrame(df_access.str.split('\t').tolist(), columns=['route_1', 'station_1', 'minuites_1', '3', 'route_2', 'station_2', 'minuites_2', '7', 'route_3', 'station_3', 'minuites_3', '11', '12'])

df_access = df_access.drop('3', axis=1)
df_access = df_access.drop('7', axis=1)
df_access = df_access.drop('11', axis=1)
df_access = df_access.drop('12', axis=1)

df_access.head()

,route_1,station_1,minuites_1,route_2,station_2,minuites_2,route_3,station_3,minuites_3
0,都営三田線,西巣鴨駅,徒歩4分,埼京線,板橋駅,徒歩14分,都電荒川線,西ケ原四丁目駅,徒歩7分
1,都営大江戸線,勝どき駅,徒歩5分,有楽町線,月島駅,徒歩9分,日比谷線,築地駅,徒歩20分
2,京王線,笹塚駅,徒歩6分,京王線,代田橋駅,徒歩7分,京王線,明大前駅,徒歩17分
3,総武線・中央線（各停）,高円寺駅,徒歩9分,丸ノ内線(池袋－荻窪),新高円寺駅,徒歩3分,丸ノ内線(池袋－荻窪),東高円寺駅,徒歩14分
4,京成金町線,京成金町駅,徒歩5分,常磐線,金町(東京都)駅,徒歩7分,京成金町線,柴又駅,徒歩17分


In [10]:
dftest_access = dftest['アクセス']

dftest_access = pd.DataFrame(dftest_access.str.split('\t').tolist(), columns=['route_1', 'station_1', 'minuites_1', '3', 'route_2', 'station_2', 'minuites_2', '7', 'route_3', 'station_3', 'minuites_3', '11', '12'])

dftest_access = dftest_access.drop('3', axis=1)
dftest_access = dftest_access.drop('7', axis=1)
dftest_access = dftest_access.drop('11', axis=1)
dftest_access = dftest_access.drop('12', axis=1)

dftest_access.head()

,route_1,station_1,minuites_1,route_2,station_2,minuites_2,route_3,station_3,minuites_3
0,東急大井町線,等々力駅,徒歩15分,東急田園都市線,桜新町駅,徒歩24分,東急東横線,自由が丘駅,徒歩27分
1,東急東横線,都立大学駅,徒歩6分,東急東横線,自由が丘駅,徒歩17分,東急大井町線,自由が丘駅,徒歩17分
2,東武東上線,北池袋駅,徒歩10分,埼京線,板橋駅,徒歩16分,山手線,池袋駅,徒歩19分
3,丸ノ内線(中野坂上－方南町),方南町駅,徒歩10分,京王線,代田橋駅,徒歩8分,京王井の頭線,明大前駅,徒歩17分
4,丸ノ内線(中野坂上－方南町),方南町駅,徒歩12分,丸ノ内線(池袋－荻窪),新高円寺駅,徒歩17分,丸ノ内線(池袋－荻窪),東高円寺駅,徒歩18分


In [0]:
route = pd.concat(objs=[df_access['route_1'], df_access['route_2']], axis=1)
route = pd.concat(objs=[df_route, df_access['route_3']], axis=1)

station = pd.concat(objs=[df_access['station_1'], df_access['station_2']], axis=1)
station = pd.concat(objs=[df_station, df_access['station_3']], axis=1)

minuites = pd.concat(objs=[df_access['minuites_1'], df_access['minuites_2']], axis=1)
minuites = pd.concat(objs=[df_minuites, df_access['minuites_3']], axis=1)

route_list = []
station_list = []
minuites_list = []

for t in set(df_route.values.flatten()):
  if isinstance(t, int) != True and isinstance(t, float) != True:
    route_list.append(t)

for elem in route_list:
  df_access[elem] = df_route.apply(lambda xs: 1 if elem in xs.values else 0, axis=1)

for t in set(df_station.values.flatten()):
  if isinstance(t, int) != True and isinstance(t, float) != True:
    station_list.append(t)

for elem in station_list:
  df_access[elem] = df_station.apply(lambda xs: 1 if elem in xs.values else 0, axis=1)

for t in set(df_minuites.values.flatten()):
  if isinstance(t, int) != True and isinstance(t, float) != True:
    minuites_list.append(t)

for elem in minuites_list:
  df_access[elem] = df_minuites.apply(lambda xs: 1 if elem in xs.values else 0, axis=1)

del route_list
del station_list
del minuites_list

# 間取り

In [11]:
df_floor_plan = pd.DataFrame(df['間取り'])

df_floor_plan.head()

,間取り
0,1K
1,1R
2,1K
3,2LDK
4,2DK


In [12]:
dftest_floor_plan = pd.DataFrame(dftest['間取り'])

dftest_floor_plan.head()

,間取り
0,2LDK
1,1R
2,1K
3,1K
4,2LDK


# 築年数

In [13]:
df_age = df['築年数']

df_age = df_age.str.extract('(.+)年(.+)ヶ月', expand=True)
df_age.columns = ['year', 'month']

df_age.head()

,year,month
0,9,9
1,44,10
2,8,6
3,29,4
4,31,7


In [14]:
dftest_age = dftest['築年数']

dftest_age = dftest_age.str.extract('(.+)年(.+)ヶ月', expand=True)
dftest_age.columns = ['year', 'month']

dftest_age.head()

,year,month
0,49,0
1,0,2
2,23,4
3,36,2
4,45,4


# 方角

In [15]:
df_direction = pd.DataFrame(df['方角'])
df_direction = df_direction.fillna(0)

df_direction.head()

,方角
0,南東
1,0
2,南
3,南
4,南


In [16]:
dftest_direction = pd.DataFrame(dftest['方角'])

dftest_direction.head()

,方角
0,南
1,南東
2,南東
3,南東
4,南


# 面積

In [17]:
df_area = df['面積']
df_area = df_area.str.extract('(.*)m2', expand=True)
df_area.columns = ['area']

df_area.head()

,area
0,20.01
1,16.5
2,22.05
3,60.48
4,39.66


In [18]:
dftest_area = dftest['面積']
dftest_area = dftest_area.str.extract('(.*)m2', expand=True)
dftest_area.columns = ['area']

dftest_area.head()

,area
0,50.22
1,20.88
2,26.93
3,23.57
4,50


# 所在階

In [19]:
df_floors = df['所在階']
df_floors = df_floors.str.extract('(.*)階／(.*)階建', expand=True)
df_floors.columns = ['floors', 'story']

df_floors.head()

,floors,story
0,1,12
1,5,10
2,12,15
3,3,4
4,1,2


In [20]:
dftest_floors = dftest['所在階']
dftest_floors = dftest_floors.str.extract('(.*)階／(.*)階建', expand=True)
dftest_floors.columns = ['floors', 'story']

dftest_floors.head()

,floors,story
0,8,8
1,3,4
2,1,4
3,1,2
4,4,4


# バス・トイレ 

In [21]:
df_bath_toilet = df['バス・トイレ']
df_bath_toilet = df_bath_toilet.str.replace('／', '', regex=False)
df_bath_toilet = df_bath_toilet.fillna('\t')
df_bath_toilet = df_bath_toilet.str.split('\t').tolist()
df_bath_toilet = pd.DataFrame(df_bath_toilet)

df_bath_toilet.head()

,0,1,2,3,4,5,6,7,8,9
0,専用バス,専用トイレ,バス・トイレ別,シャワー,浴室乾燥機,,温水洗浄便座,None,None,None
1,専用トイレ,シャワー,温水洗浄便座,None,None,None,None,None,None,None
2,専用バス,専用トイレ,バス・トイレ別,シャワー,浴室乾燥機,,温水洗浄便座,洗面台独立,脱衣所,None
3,専用バス,専用トイレ,バス・トイレ別,シャワー,温水洗浄便座,洗面台独立,None,None,None,None
4,専用バス,専用トイレ,バス・トイレ別,シャワー,追焚機能,None,None,None,None,None


In [22]:
dftest_bath_toilet = dftest['バス・トイレ']
dftest_bath_toilet = dftest_bath_toilet.str.replace('／', '', regex=False)
dftest_bath_toilet = dftest_bath_toilet.fillna('\t')
dftest_bath_toilet = dftest_bath_toilet.str.split('\t').tolist()
dftest_bath_toilet = pd.DataFrame(dftest_bath_toilet)

dftest_bath_toilet.head()

,0,1,2,3,4,5,6,7,8,9
0,専用バス,専用トイレ,バス・トイレ別,シャワー,追焚機能,温水洗浄便座,洗面台独立,脱衣所,None,None
1,専用バス,専用トイレ,バス・トイレ別,シャワー,浴室乾燥機,,温水洗浄便座,洗面台独立,脱衣所,None
2,専用バス,専用トイレ,バス・トイレ別,シャワー,温水洗浄便座,None,None,None,None,None
3,シャワー,None,None,None,None,None,None,None,None,None
4,専用バス,専用トイレ,バス・トイレ別,None,None,None,None,None,None,None


# キッチン

In [23]:
df_kitchen = df['キッチン']
df_kitchen = df_kitchen.str.replace('／', '', regex=False)
df_kitchen = df_kitchen.fillna('\t')
df_kitchen = pd.DataFrame(df_kitchen.str.split('\t').tolist())

df_kitchen.head()

,0,1,2,3,4,5,6,7,8,9
0,ガスコンロ,コンロ2口,システムキッチン,,給湯,None,None,None,None,None
1,ガスコンロ,システムキッチン,,給湯,None,None,None,None,None,None
2,ガスコンロ,コンロ2口,システムキッチン,,給湯,None,None,None,None,None
3,ガスコンロ,給湯,None,None,None,None,None,None,None,None
4,給湯,独立キッチン,None,None,None,None,None,None,None,None


In [24]:
dftest_kitchen = dftest['キッチン']
dftest_kitchen = dftest_kitchen.str.replace('／', '', regex=False)
dftest_kitchen = dftest_kitchen.fillna('\t')
dftest_kitchen = pd.DataFrame(dftest_kitchen.str.split('\t').tolist())

dftest_kitchen.head()

,0,1,2,3,4,5,6,7,8,9
0,ガスコンロ,コンロ3口,システムキッチン,,給湯,L字キッチン,None,None,None,None
1,ガスコンロ,コンロ2口,システムキッチン,,給湯,None,None,None,None,None
2,ガスコンロ,給湯,None,None,None,None,None,None,None,None
3,ガスコンロ,給湯,None,None,None,None,None,None,None,None
4,コンロ設置可（口数不明）,給湯,None,None,None,None,None,None,None,None


# 放送・通信

In [25]:
df_broadcast = df['放送・通信']
df_broadcast = df_broadcast.str.replace('／', '', regex=False)
df_broadcast = df_broadcast.fillna('\t')
df_broadcast = pd.DataFrame(df_broadcast.str.split('\t').tolist())

df_broadcast.head()

,0,1,2,3,4,5,6
0,インターネット対応,CATV,CSアンテナ,BSアンテナ,None,None,None
1,インターネット対応,None,None,None,None,None,None
2,インターネット対応,光ファイバー,CSアンテナ,BSアンテナ,None,None,None
3,インターネット対応,光ファイバー,None,None,None,None,None
4,,,None,None,None,None,None


In [26]:
dftest_broadcast = dftest['放送・通信']
dftest_broadcast = dftest_broadcast.str.replace('／', '', regex=False)
dftest_broadcast = dftest_broadcast.fillna('\t')
dftest_broadcast = pd.DataFrame(dftest_broadcast.str.split('\t').tolist())

dftest_broadcast.head()

,0,1,2,3,4,5,6
0,インターネット対応,None,None,None,None,None,None
1,インターネット対応,CSアンテナ,BSアンテナ,インターネット使用料無料,None,None,None
2,インターネット対応,光ファイバー,CATV,None,None,None,None
3,,,None,None,None,None,None
4,インターネット対応,None,None,None,None,None,None


# 室内設備

In [27]:
df_indoor = df['室内設備']
df_indoor = df_indoor.str.replace('／', '', regex=False)
df_indoor = df_indoor.fillna('\t')
df_indoor = pd.DataFrame(df_indoor.str.split('\t').tolist())

df_indoor.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,エアコン付,シューズボックス,バルコニー,フローリング,室内洗濯機置場,敷地内ごみ置き場,,エレベーター,公営水道,下水,都市ガス,None,None,None,None,None,None,None,None,None,None,None,None
1,エアコン付,シューズボックス,バルコニー,フローリング,室内洗濯機置場,敷地内ごみ置き場,,エレベーター,タイル張り,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,エアコン付,ウォークインクローゼット,シューズボックス,バルコニー,フローリング,室内洗濯機置場,敷地内ごみ置き場,,エレベーター,タイル張り,公営水道,下水,都市ガス,None,None,None,None,None,None,None,None,None,None
3,エアコン付,シューズボックス,バルコニー,2面採光,室内洗濯機置場,エレベーター,公営水道,下水,都市ガス,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,バルコニー,フローリング,室内洗濯機置場,公営水道,下水,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [28]:
dftest_indoor = dftest['室内設備']
dftest_indoor = dftest_indoor.str.replace('／', '', regex=False)
dftest_indoor = dftest_indoor.fillna('\t')
dftest_indoor = pd.DataFrame(dftest_indoor.str.split('\t').tolist())

dftest_indoor.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,エアコン付,シューズボックス,バルコニー,フローリング,エレベーター,公営水道,下水,都市ガス,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,エアコン付,シューズボックス,フローリング,室内洗濯機置場,公営水道,下水,都市ガス,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,エアコン付,シューズボックス,バルコニー,フローリング,2面採光,室内洗濯機置場,タイル張り,公営水道,下水,都市ガス,None,None,None,None,None,None,None,None,None,None,None,None,None
3,エアコン付,シューズボックス,2面採光,室外洗濯機置場,公営水道,下水,都市ガス,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,冷房,エアコン付,シューズボックス,バルコニー,フローリング,室内洗濯機置場,敷地内ごみ置き場,公営水道,下水,都市ガス,None,None,None,None,None,None,None,None,None,None,None,None,None


# 駐車場

In [29]:
df_parking =df['駐車場']
df_parking = df_parking.fillna('\t')
df_parking = pd.DataFrame(df_parking.str.split('\t').tolist())

df_parking.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,駐輪場,空有,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,駐輪場,空有,駐車場,無,バイク置き場,無,None,None,None,None,None,None,None,None,None,None,None,None
2,駐輪場,空有,バイク置き場,空有,駐車場,近隣,"30,000円(税込)",距離100m,None,None,None,None,None,None,None,None,None,None
3,駐車場,無,駐輪場,無,バイク置き場,無,None,None,None,None,None,None,None,None,None,None,None,None
4,駐車場,近隣,"17,000円(税込)",距離300m,駐輪場,無,バイク置き場,無,None,None,None,None,None,None,None,None,None,None


In [30]:
dftest_parking =dftest['駐車場']
dftest_parking = dftest_parking.fillna('\t')
dftest_parking = pd.DataFrame(dftest_parking.str.split('\t').tolist())

dftest_parking.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,駐輪場,空有,駐車場,無,バイク置き場,無,None,None,None,None,None,None,None,None,None,None,None,None,None
1,駐車場,近隣,"34,000円(税込)",距離200m,駐輪場,無,バイク置き場,無,None,None,None,None,None,None,None,None,None,None,None
2,駐車場,空有,"5,000円(税込)",(敷金あり,1ヶ月),駐輪場,空有,バイク置き場,空有,None,None,None,None,None,None,None,None,None,None
3,駐輪場,空有,0円,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,駐車場,近隣,"22,600円",距離177m,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


# 周辺環境

In [6]:
df_surrounding = df['周辺環境']
df_surrounding = df_surrounding.fillna('\t')
df_surrounding = pd.DataFrame(df_surrounding.str.split('\t').tolist())
df_surrounding.columns = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09']

df_surrounding = df_surrounding.fillna('【】')

for i in range(len(df_surrounding.columns)):
  df_temp = df_surrounding.iloc[:, i]
  df_surrounding = df_surrounding.drop('0'+str(i), axis=1)
  df_surrounding = pd.concat(objs=[df_surrounding, pd.DataFrame(df_temp.str.extract('【(.*)】(.*)m', expand=True))], axis=1)
  del df_temp

df_surrounding.head()

20

In [7]:
dftest_surrounding = dftest['周辺環境']
dftest_surrounding = dftest_surrounding.fillna('\t')
dftest_surrounding = pd.DataFrame(dftest_surrounding.str.split('\t').tolist())
dftest_surrounding.columns = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09']

dftest_surrounding = dftest_surrounding.fillna('【】')

for i in range(len(dftest_surrounding.columns)):
  dftest_temp = dftest_surrounding.iloc[:, i]
  dftest_surrounding = dftest_surrounding.drop('0'+str(i), axis=1)
  dftest_surrounding = pd.concat(objs=[dftest_surrounding, pd.DataFrame(dftest_temp.str.extract('【(.*)】(.*)m', expand=True))], axis=1)
  del dftest_temp

dftest_surrounding.head()

20

# 建物構造

In [33]:
df_structure = pd.DataFrame(df['建物構造'])

df_structure.head()

,建物構造
0,RC（鉄筋コンクリート）
1,鉄骨造
2,RC（鉄筋コンクリート）
3,RC（鉄筋コンクリート）
4,木造


In [34]:
dftest_structure = pd.DataFrame(dftest['建物構造'])

dftest_structure.head()

,建物構造
0,鉄骨造
1,RC（鉄筋コンクリート）
2,鉄骨造
3,木造
4,RC（鉄筋コンクリート）


# 契約期間

In [35]:
df_constract = pd.DataFrame(df['契約期間'])

df_constract.head()

,契約期間
0,2年間
1,2年間
2,2年間
3,2年間\t※この物件は\t定期借家\tです。
4,2年間


In [36]:
dftest_constract = pd.DataFrame(dftest['契約期間'])

dftest_constract.head()

,契約期間
0,NaN
1,2年間
2,2年間
3,2年間
4,2年間


# Join all DataFrame

In [0]:
df_x = pd.concat(objs=[df_x, df_location], axis=1)
df_x = pd.concat(objs=[df_x, df_access], axis=1)
df_x = pd.concat(objs=[df_x, df_floor_plan], axis=1)
df_x = pd.concat(objs=[df_x, df_age], axis=1)
df_x = pd.concat(objs=[df_x, df_direction], axis=1)
df_x = pd.concat(objs=[df_x, df_area], axis=1)
df_x = pd.concat(objs=[df_x, df_floors], axis=1)
df_x = pd.concat(objs=[df_x, df_bath_toilet], axis=1)
df_x = pd.concat(objs=[df_x, df_kitchen], axis=1)
df_x = pd.concat(objs=[df_x, df_broadcast], axis=1)
df_x = pd.concat(objs=[df_x, df_indoor], axis=1)
df_x = pd.concat(objs=[df_x, df_parking], axis=1)
df_x = pd.concat(objs=[df_x, df_surrounding], axis=1)
df_x = pd.concat(objs=[df_x, df_structure], axis=1)
df_x = pd.concat(objs=[df_x, df_constract], axis=1)

dftest_x = pd.concat(objs=[dftest_x, dftest_location], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_access], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_floor_plan], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_age], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_direction], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_area], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_floors], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_bath_toilet], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_kitchen], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_broadcast], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_indoor], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_parking], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_surrounding], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_structure], axis=1)
dftest_x = pd.concat(objs=[dftest_x, dftest_constract], axis=1)

In [0]:
common_col = (df_x.columns & dftest_x.columns)

df_x = df_x[common_col]
dftest_x = dftest_x[common_col]
    
print(df_x.shape)
print(dftest_x.shape)

# Onehot Encoding

In [0]:
onehot = []
for name, type in zip(df_x.columns, df_x.dtypes):
  if type == 'object':
    onehot.append(name)
  
df_x = pd.get_dummies(df_x, prefix=onehot)
dftest_x = pd.get_dummies(dftest_x, prefix=onehot)

# from object to int

In [0]:
from sklearn.preprocessing import LabelEncoder

cat_features = []
onehot = []

for i, j in zip(df_x.dtypes, df_x.columns):
  if i == 'object':
    cat_features.append(j)
    onehot.append(j)

for col in cat_features:
  lbl = LabelEncoder()
  df_x[col] = lbl.fit_transform(list(df_x[col].values))
  dftest_x[col] = lbl.fit_transform(list(dftest_x[col].values))

# interpolate missing value

In [0]:
df_x = df_x.fillna(df_x.mean())
dftest_x = dftest_x.fillna(dftest_x.mean())

# make Model

In [0]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import preprocessing, linear_model, svm
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

# モデルの作成
mod = lgb.LGBMRegressor()

parameters = {'max_depth': [15, 16, 17, 18, 19, 20],
              'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1],
              'n_estimators': [100, 1000, 5000, 10000, 50000],}

model = GridSearchCV(mod,
                     parameters,
                     cv=5)

# 説明変数の設定
X = df_x.as_matrix()

# 目的変数の設定
Y = df_t.as_matrix()

# 学習
model.fit(X, Y)

# predict

In [0]:
#予測
XX = dftest_x.as_matrix()

YY = model.predict(XX)

# 予測データ(df_pre)
dftest_t = pd.DataFrame(YY)

# 提出用に変換
dftest_t = dftest_t.astype(int)
dftest_t = dftest_t.round()
dftest_t = pd.concat(objs=[dftest['id'], dftest_t], axis=1)
dftest_t.head()

In [0]:
# tsvファイル作成
dftest_t.to_csv('dftest_t.csv', sep='\t', index=False, header=False)